In [1]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

!whoami
!date

alibow
Mon Jun 29 10:41:45 PDT 2020


In [2]:

output_dirs = ['/share/costeffectiveness/results/vivarium_conic_lsff/india/2020_06_26_20_35_00/count_data/']

locations = ['India']

# NOTE: Conclusion statements in this notebook have not been updated

# 1. Iron effect on birth weight

In [3]:
births = pd.read_hdf(output_dirs[0] + 'births.hdf')
births.head()

# note: no stratification of birth counts by iron coverage group

,year,sex,folic_acid_fortification_group,measure,input_draw,scenario,value
0,2020,female,covered,live_births,21,baseline,1857.0
1,2020,female,covered,live_births,21,baseline,12.0
2,2020,female,covered,live_births,21,baseline,530.0
3,2020,female,covered,live_births,21,folic_acid_fortification_scale_up,1857.0
4,2020,female,covered,live_births,21,folic_acid_fortification_scale_up,12.0


In [4]:
bw = pd.read_hdf(output_dirs[0] + 'birth_weight.hdf')
bw.head()

,year,sex,measure,input_draw,scenario,value,iron_fortification_group
0,2020,female,birth_weight_mean,21,baseline,2912.186341,uncovered
1,2020,female,birth_weight_mean,21,baseline,2897.547480,covered
2,2020,female,birth_weight_mean,21,folic_acid_fortification_scale_up,2912.186341,uncovered
3,2020,female,birth_weight_mean,21,folic_acid_fortification_scale_up,2897.547480,covered
4,2020,female,birth_weight_mean,21,iron_folic_acid_fortification_scale_up,2912.186341,uncovered


In [5]:
bw_by_coverage_and_scenario = bw.loc[bw.measure == 'birth_weight_mean']
bw_by_coverage_and_scenario = bw_by_coverage_and_scenario.groupby(['scenario','iron_fortification_group']).mean()
bw_by_coverage_and_scenario

# we would expect mean birth weight to be higher in covered group than uncovered group (for all scenarios)
# we are seeing the opposite
# but Kjell verified that the MEDIAN birth weight in covered group > median birth weight in uncovered group
    # this suggests that the distribution is differentially skewed high
    
# we would also expect to see simular mean birth weights by coverage group between scenarios
    # we are not seeing this.

input_draw  \
scenario                               iron_fortification_group               
baseline                               covered                       437.52   
                                       uncovered                     437.52   
folic_acid_fortification_scale_up      covered                       437.52   
                                       uncovered                     437.52   
iron_folic_acid_fortification_scale_up covered                       437.52   
                                       uncovered                     437.52   
vitamin_a_fortification_scale_up       covered                       437.52   
                                       uncovered                     437.52   

                                                                       value  
scenario                               iron_fortification_group               
baseline                               covered                   2884.206771  
                                       uncovered                 2884.756851  
folic_acid_fortification_scale_up      covered                   2884.206771  
                                       uncovered                 2884.756851  
iron_folic_acid_fortification_scale_up covered                   2884.345141  
                                       uncovered                 2884.740523  
vitamin_a_fortification_scale_up       covered                   2884.206771  
                                       uncovered                 2884.756851

## Birth weight conclusions

NO CHANGE IN BIRTH WEIGHT IN IRON SCENARIO OR BY COVERAGE GROUP

# 2. Vitamin A Fortification and Vitamin A Deficiency Prevalence

In [6]:
state_pt = pd.read_hdf(output_dirs[0] + 'state_person_time.hdf')
state_pt.head()

,year,age_group,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,diarrheal_diseases,covered,covered,person_time,21,baseline,0.0
1,2020,1_to_4,diarrheal_diseases,covered,covered,person_time,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,diarrheal_diseases,covered,covered,person_time,21,iron_folic_acid_fortification_scale_up,0.0
3,2020,1_to_4,diarrheal_diseases,covered,covered,person_time,21,vitamin_a_fortification_scale_up,0.0
4,2020,1_to_4,diarrheal_diseases,covered,covered,person_time,29,baseline,0.0


In [7]:
def calculate_stratified_vad_prevalence(strata_cols):
    state_pt = pd.read_hdf(output_dirs[0] + 'state_person_time.hdf')
    vad = state_pt.loc[state_pt.cause == 'vitamin_a_deficiency']
    vad = vad.groupby((['scenario','input_draw'] + strata_cols)).sum()
    pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')
    pt = pt.groupby((['scenario','input_draw'] + strata_cols)).sum()
    vad_prev = vad / pt
    vad_prev = vad_prev.reset_index()
    vad_prev = vad_prev.loc[vad_prev.scenario != 'iron_fortification_scale_up']
    vad_prev = (vad_prev.groupby((['scenario'] + strata_cols)).mean()
                .rename(columns={'value':'vad_prevalence'})
                .drop(columns='input_draw'))
    return vad_prev

In [8]:
overall = calculate_stratified_vad_prevalence([])
overall

# looks good! (Prevalence of VAD is lower in vitamin_a_fortification_scale_up scenario than baseline scenario)

,vad_prevalence
scenario,
baseline,0.001415
folic_acid_fortification_scale_up,0.001415
iron_folic_acid_fortification_scale_up,0.001415
vitamin_a_fortification_scale_up,0.001125


In [9]:
by_year = calculate_stratified_vad_prevalence(['year'])
by_year

# looks good! 
# VAD prevalence is the same in each scenario in 2020, and then lower in vit_a_scenaario in 2021-2025

vad_prevalence
scenario                               year                
baseline                               2020        0.000626
                                       2021        0.001209
                                       2022        0.003282
                                       2023        0.023188
folic_acid_fortification_scale_up      2020        0.000626
                                       2021        0.001209
                                       2022        0.003282
                                       2023        0.023188
iron_folic_acid_fortification_scale_up 2020        0.000626
                                       2021        0.001209
                                       2022        0.003282
                                       2023        0.023188
vitamin_a_fortification_scale_up       2020        0.000626
                                       2021        0.000990
                                       2022        0.002221
                                       2023        0.015609

In [10]:
by_age = calculate_stratified_vad_prevalence(['age_group'])
by_age

# no change in neonatal age groups between scenarios, as expected
# lower VAD prevalence in vitamin_a_scenario in post_neonatal and 1_to_4 ages, as expected
# looks good!

vad_prevalence
scenario                               age_group                     
baseline                               1_to_4                0.000976
                                       early_neonatal        0.050028
                                       late_neonatal         0.088844
                                       post_neonatal         0.099297
folic_acid_fortification_scale_up      1_to_4                0.000976
                                       early_neonatal        0.050028
                                       late_neonatal         0.088844
                                       post_neonatal         0.099297
iron_folic_acid_fortification_scale_up 1_to_4                0.000976
                                       early_neonatal        0.050034
                                       late_neonatal         0.088846
                                       post_neonatal         0.099297
vitamin_a_fortification_scale_up       1_to_4                0.000737
                                       early_neonatal        0.050353
                                       late_neonatal         0.089950
                                       post_neonatal         0.087606

In [11]:
by_coverage_group = calculate_stratified_vad_prevalence(['vitamin_a_fortification_group'])
by_coverage_group

# unexpected results here...

# we should expect...

    # all VAD_prev < 1 in each coverage group
    # covered VAD_prev ~= uncovered VAD_prev
    # effectively_covered VAD_prev * 2.22 ~= uncovered VAD_prev in both scenarios
    # all VAD_prev by coverage group in baseline scenario should ~= VAD_prev by coverage group in vitamin_a scenario

vad_prevalence
scenario                               vitamin_a_fortification_group                
baseline                               covered                              0.198401
                                       effectively_covered                  0.000656
                                       uncovered                            0.001585
folic_acid_fortification_scale_up      covered                              0.198401
                                       effectively_covered                  0.000656
                                       uncovered                            0.001585
iron_folic_acid_fortification_scale_up covered                              0.198413
                                       effectively_covered                  0.000656
                                       uncovered                            0.001585
vitamin_a_fortification_scale_up       covered                              0.048683
                                       effectively_covered                  0.000553
                                       uncovered                            0.002080

## Conclusions

- For the covered group, VAD state person time > overall person time, which appears to be an error (prevalence > 1)
- For the baseline scenario, VAD prevalence in the effectively covered group is ~ 0.45 times that in the uncovered group, as expected. However, in the vitamin A scenario this is not the case and the difference is greater between these groups -- we would expect to see the same rates here and just more person time in the covered groups in the vitamin A scenario.
- Otherwise, behavior by year, age group, and overall looks as expected
- VAD prevalence does not decrease until 2021, indicating that the coverage algorithm was implemented correctly

# 3. Iron effect on hemoglobin

In [12]:
hb = pd.read_hdf(output_dirs[0] + 'hemoglobin_level.hdf')
hb.head()

,sex,measure,input_draw,scenario,value,age,status,responsive
0,female,hemoglobin_mean,21,baseline,104.391697,0.5,covered,responsive
1,female,hemoglobin_mean,21,baseline,84.403095,0.5,covered,non-responsive
2,female,hemoglobin_mean,21,baseline,104.741896,0.5,uncovered,responsive
3,female,hemoglobin_mean,21,baseline,95.027138,0.5,uncovered,non-responsive
4,female,hemoglobin_mean,21,baseline,106.950925,1,covered,responsive


In [13]:
np.unique(hb['measure'])

array(['hemoglobin_mean', 'hemoglobin_variance'], dtype=object)

In [14]:
pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')
pt.head()

# NOTE: cannot calculate weighted averages of Hb because persontime data not stratified by iron coverage or responsiveness

,year,age_group,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,covered,covered,person_time,21,baseline,0.0
1,2020,1_to_4,covered,covered,person_time,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,covered,covered,person_time,21,iron_folic_acid_fortification_scale_up,0.0
3,2020,1_to_4,covered,covered,person_time,21,vitamin_a_fortification_scale_up,0.0
4,2020,1_to_4,covered,covered,person_time,29,baseline,0.0


In [15]:
def get_stratified_hb_mean(strata_cols):
    hb = pd.read_hdf(output_dirs[0] + 'hemoglobin_level.hdf')
    hb = hb.loc[hb.scenario != 'vitamin_a_fortification_scale_up']
    hb = hb.loc[hb.measure == 'hemoglobin_mean']
    hb = hb.groupby(['scenario','measure'] + strata_cols).mean()
    return hb

In [16]:
overall = get_stratified_hb_mean([])
overall


# looks good! (mean hemoglobin is slightly higher in iron_fortification scenario)

,,input_draw,value
scenario,measure,,
baseline,hemoglobin_mean,437.52,98.649279
folic_acid_fortification_scale_up,hemoglobin_mean,437.52,98.649278
iron_folic_acid_fortification_scale_up,hemoglobin_mean,437.52,99.710850


In [17]:
by_coverage = get_stratified_hb_mean(['status'])
by_coverage

# hemoglobin is lower in covered group, which is the opposite of what we would expect

# should expect...
    
    # covered mean hemoglobin to be higher than uncovered mean hemoglobin in both scenarios
        # note, if this is not true, it should be true for MEDIAN hemoglobin by coverage group
    # mean hemoglobin by coverage group should be approximately equal between scenarios
        # note, it is ok if covered hemoglobin in the iron scenario is slightly less than the covered 
        # group in the baseline scenario

input_draw  \
scenario                               measure         status                  
baseline                               hemoglobin_mean covered        437.52   
                                                       uncovered      437.52   
folic_acid_fortification_scale_up      hemoglobin_mean covered        437.52   
                                                       uncovered      437.52   
iron_folic_acid_fortification_scale_up hemoglobin_mean covered        437.52   
                                                       uncovered      437.52   

                                                                       value  
scenario                               measure         status                 
baseline                               hemoglobin_mean covered     96.464106  
                                                       uncovered  100.834452  
folic_acid_fortification_scale_up      hemoglobin_mean covered     96.464106  
                                                       uncovered  100.834451  
iron_folic_acid_fortification_scale_up hemoglobin_mean covered     98.603525  
                                                       uncovered  100.818176

In [18]:
by_responsiveness = get_stratified_hb_mean(['status','responsive'])
by_responsiveness

# we are seeing some unexpected results here

    
# expected results:

    # for a given scenario...
        # covered, responsive hemoglobin > uncovered, responsive hemoglobin (by ~ 3 units)
        # covered, non-responsive hemoglobin ~= uncovered, non-responsive hemoglobin
    
    # mean hemoglobin by coverage group and responsiveness should be approximately equal between scenarios
        
    # for a given coverage group, responsive hemoglobin > non-responsive hemoglobin
        # for a given ANEMIA group (i.e. severe anemia)... responsive hemoglobin ~= non-responsive hemoglobin
            # can verify this in interactive sim?
        # note: difference between responsive and non-responsive should be larger in covered group than uncovered group

input_draw  \
scenario                               measure         status    responsive                   
baseline                               hemoglobin_mean covered   non-responsive      437.52   
                                                                 responsive          437.52   
                                                       uncovered non-responsive      437.52   
                                                                 responsive          437.52   
folic_acid_fortification_scale_up      hemoglobin_mean covered   non-responsive      437.52   
                                                                 responsive          437.52   
                                                       uncovered non-responsive      437.52   
                                                                 responsive          437.52   
iron_folic_acid_fortification_scale_up hemoglobin_mean covered   non-responsive      437.52   
                                                                 responsive          437.52   
                                                       uncovered non-responsive      437.52   
                                                                 responsive          437.52   

                                                                                      value  
scenario                               measure         status    responsive                  
baseline                               hemoglobin_mean covered   non-responsive   86.381271  
                                                                 responsive      106.546940  
                                                       uncovered non-responsive   96.277177  
                                                                 responsive      105.391726  
folic_acid_fortification_scale_up      hemoglobin_mean covered   non-responsive   86.381271  
                                                                 responsive      106.546940  
                                                       uncovered non-responsive   96.277177  
                                                                 responsive      105.391725  
iron_folic_acid_fortification_scale_up hemoglobin_mean covered   non-responsive   90.530502  
                                                                 responsive      106.676547  
                                                       uncovered non-responsive   96.245603  
                                                                 responsive      105.390749

In [19]:
by_age = get_stratified_hb_mean(['age'])
by_age

# would expect to see slight difference for 0.5 age group between scenarios
# expect to see larger differences between scenarios for other ages

input_draw  \
scenario                               measure         age               
baseline                               hemoglobin_mean 0.5      437.52   
                                                       1        437.52   
                                                       2        437.52   
                                                       3        437.52   
folic_acid_fortification_scale_up      hemoglobin_mean 0.5      437.52   
                                                       1        437.52   
                                                       2        437.52   
                                                       3        437.52   
iron_folic_acid_fortification_scale_up hemoglobin_mean 0.5      437.52   
                                                       1        437.52   
                                                       2        437.52   
                                                       3        437.52   

                                                                 value  
scenario                               measure         age              
baseline                               hemoglobin_mean 0.5   96.074232  
                                                       1     99.190200  
                                                       2     99.625878  
                                                       3     99.706804  
folic_acid_fortification_scale_up      hemoglobin_mean 0.5   96.074232  
                                                       1     99.190200  
                                                       2     99.625878  
                                                       3     99.706804  
iron_folic_acid_fortification_scale_up hemoglobin_mean 0.5   97.460305  
                                                       1    100.319010  
                                                       2    100.340956  
                                                       3    100.723130

In [20]:
by_responsiveness = get_stratified_hb_mean(['responsive'])
by_responsiveness

# we would expect...

    # baseline non-responsive hemoglobin ~= iron scenario non-responsive hemoglobin
    # baseline responsive hemoglobin < iron scenario responsive hemoglobin (by < 3 units)
    
# we are seeing unexpected results here in that ...

    # baseline non-responsive hemoglobin < iron scenario non-responsive hemoglobin
    # baseline responsive hemoglobin = iron scenario responsive hemoglobin

input_draw  \
scenario                               measure         responsive                   
baseline                               hemoglobin_mean non-responsive      437.52   
                                                       responsive          437.52   
folic_acid_fortification_scale_up      hemoglobin_mean non-responsive      437.52   
                                                       responsive          437.52   
iron_folic_acid_fortification_scale_up hemoglobin_mean non-responsive      437.52   
                                                       responsive          437.52   

                                                                            value  
scenario                               measure         responsive                  
baseline                               hemoglobin_mean non-responsive   91.329224  
                                                       responsive      105.969333  
folic_acid_fortification_scale_up      hemoglobin_mean non-responsive   91.329224  
                                                       responsive      105.969333  
iron_folic_acid_fortification_scale_up hemoglobin_mean non-responsive   93.388053  
                                                       responsive      106.033648

## Conclusions

- hemoglobin is lower in covered group, which is the opposite of what we would expect
- there is significant variation in mean hemoglobin between responsive and non-responsive groups in the uncovered population, which is unexpected
- seeing a difference in mean hemoglobin between scenarios for the 0.5 age, which is unexpected
- seeing an increase in mean hemoglobin between scenarios in the non-responsive population, which is unexpected

# 4. Iron fortification on anemia prevalence

In [21]:
anemia_pt = pd.read_hdf(output_dirs[0] + 'anemia_state_person_time.hdf')
anemia_pt.head()

,year,age_group,sex,measure,input_draw,scenario,value
0,2020,1_to_4,female,anemia_mild,21,baseline,44605.015743
1,2020,1_to_4,female,anemia_mild,21,folic_acid_fortification_scale_up,44605.015743
2,2020,1_to_4,female,anemia_mild,21,iron_folic_acid_fortification_scale_up,44605.015743
3,2020,1_to_4,female,anemia_mild,21,vitamin_a_fortification_scale_up,44605.015743
4,2020,1_to_4,female,anemia_mild,29,baseline,43965.626283


In [22]:
def get_anemia_prevalence(strata_cols):
    anemia_pt = pd.read_hdf(output_dirs[0] + 'anemia_state_person_time.hdf')
    anemia_pt = anemia_pt.loc[anemia_pt.scenario != 'vitamin_a_fortification_scale_up']
    anemia_pt = anemia_pt.groupby((['scenario','input_draw'] + strata_cols + ['measure'])).sum()
    pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')
    pt = pt.loc[pt.scenario != 'vitamin_a_fortification_scale_up']
    pt = pt.groupby((['scenario','input_draw'] + strata_cols)).sum()
    anemia_prev = anemia_pt / pt
    anemia_prev = anemia_prev.groupby((['scenario','measure'] + strata_cols)).mean() 
    return anemia_prev

In [23]:
overall = get_anemia_prevalence([])
overall

# we are seeing a VERY slight decrease in anemia prevalence between scenarios
# this is generally expected
# should see decrease in all anemia prevalences in iron scenario relative to baseline
    # an increase in anemia_none prevalence

value
scenario                               measure                  
baseline                               anemia_mild      0.001365
                                       anemia_moderate  0.001482
                                       anemia_none      0.001865
                                       anemia_severe    0.000076
folic_acid_fortification_scale_up      anemia_mild      0.001365
                                       anemia_moderate  0.001482
                                       anemia_none      0.001865
                                       anemia_severe    0.000076
iron_folic_acid_fortification_scale_up anemia_mild      0.001361
                                       anemia_moderate  0.001472
                                       anemia_none      0.001879
                                       anemia_severe    0.000076

In [24]:
by_year = get_anemia_prevalence(['year'])
by_year = by_year.reset_index().sort_values(by=['measure','year','scenario']).set_index(['measure','year','scenario'])
by_year

# this is displaying expected behavior
# no difference in 2020, slight differences in later years

value
measure         year scenario                                        
anemia_mild     2020 baseline                                0.000607
                     folic_acid_fortification_scale_up       0.000607
                     iron_folic_acid_fortification_scale_up  0.000607
                2021 baseline                                0.001167
                     folic_acid_fortification_scale_up       0.001167
                     iron_folic_acid_fortification_scale_up  0.001166
                2022 baseline                                0.003158
                     folic_acid_fortification_scale_up       0.003158
                     iron_folic_acid_fortification_scale_up  0.003144
                2023 baseline                                0.022264
                     folic_acid_fortification_scale_up       0.022264
                     iron_folic_acid_fortification_scale_up  0.022149
anemia_moderate 2020 baseline                                0.000658
                     folic_acid_fortification_scale_up       0.000658
                     iron_folic_acid_fortification_scale_up  0.000658
                2021 baseline                                0.001266
                     folic_acid_fortification_scale_up       0.001266
                     iron_folic_acid_fortification_scale_up  0.001261
                2022 baseline                                0.003432
                     folic_acid_fortification_scale_up       0.003432
                     iron_folic_acid_fortification_scale_up  0.003397
                2023 baseline                                0.024229
                     folic_acid_fortification_scale_up       0.024229
                     iron_folic_acid_fortification_scale_up  0.023926
anemia_none     2020 baseline                                0.000828
                     folic_acid_fortification_scale_up       0.000828
                     iron_folic_acid_fortification_scale_up  0.000828
                2021 baseline                                0.001597
                     folic_acid_fortification_scale_up       0.001597
                     iron_folic_acid_fortification_scale_up  0.001604
                2022 baseline                                0.004315
                     folic_acid_fortification_scale_up       0.004315
                     iron_folic_acid_fortification_scale_up  0.004365
                2023 baseline                                0.030401
                     folic_acid_fortification_scale_up       0.030401
                     iron_folic_acid_fortification_scale_up  0.030840
anemia_severe   2020 baseline                                0.000034
                     folic_acid_fortification_scale_up       0.000034
                     iron_folic_acid_fortification_scale_up  0.000034
                2021 baseline                                0.000065
                     folic_acid_fortification_scale_up       0.000065
                     iron_folic_acid_fortification_scale_up  0.000065
                2022 baseline                                0.000177
                     folic_acid_fortification_scale_up       0.000177
                     iron_folic_acid_fortification_scale_up  0.000175
                2023 baseline                                0.001253
                     folic_acid_fortification_scale_up       0.001253
                     iron_folic_acid_fortification_scale_up  0.001234

In [25]:
by_age = get_anemia_prevalence(['age_group'])
by_age = by_age.reset_index().sort_values(by=['measure','age_group','scenario']).set_index(['measure','age_group','scenario'])
by_age

# this is showing expected behavior... no change in early/late neonatal age groups, but slight change in others

value
measure         age_group      scenario                                        
anemia_mild     1_to_4         baseline                                0.001116
                               folic_acid_fortification_scale_up       0.001116
                               iron_folic_acid_fortification_scale_up  0.001112
                early_neonatal baseline                                0.005111
                               folic_acid_fortification_scale_up       0.005111
                               iron_folic_acid_fortification_scale_up  0.005111
                late_neonatal  baseline                                0.021819
                               folic_acid_fortification_scale_up       0.021819
                               iron_folic_acid_fortification_scale_up  0.021819
                post_neonatal  baseline                                0.061953
                               folic_acid_fortification_scale_up       0.061953
                               iron_folic_acid_fortification_scale_up  0.061949
anemia_moderate 1_to_4         baseline                                0.001112
                               folic_acid_fortification_scale_up       0.001112
                               iron_folic_acid_fortification_scale_up  0.001102
                early_neonatal baseline                                0.042943
                               folic_acid_fortification_scale_up       0.042943
                               iron_folic_acid_fortification_scale_up  0.042948
                late_neonatal  baseline                                0.089761
                               folic_acid_fortification_scale_up       0.089761
                               iron_folic_acid_fortification_scale_up  0.089762
                post_neonatal  baseline                                0.082260
                               folic_acid_fortification_scale_up       0.082260
                               iron_folic_acid_fortification_scale_up  0.082199
anemia_none     1_to_4         baseline                                0.001601
                               folic_acid_fortification_scale_up       0.001601
                               iron_folic_acid_fortification_scale_up  0.001614
                early_neonatal baseline                                0.000356
                               folic_acid_fortification_scale_up       0.000356
                               iron_folic_acid_fortification_scale_up  0.000356
                late_neonatal  baseline                                0.000714
                               folic_acid_fortification_scale_up       0.000714
                               iron_folic_acid_fortification_scale_up  0.000714
                post_neonatal  baseline                                0.069172
                               folic_acid_fortification_scale_up       0.069172
                               iron_folic_acid_fortification_scale_up  0.069245
anemia_severe   1_to_4         baseline                                0.000049
                               folic_acid_fortification_scale_up       0.000049
                               iron_folic_acid_fortification_scale_up  0.000048
                early_neonatal baseline                                0.014476
                               folic_acid_fortification_scale_up       0.014476
                               iron_folic_acid_fortification_scale_up  0.014478
                late_neonatal  baseline                                0.005497
                               folic_acid_fortification_scale_up       0.005497
                               iron_folic_acid_fortification_scale_up  0.005497
                post_neonatal  baseline                                0.005357
                               folic_acid_fortification_scale_up       0.005357
                               iron_folic_acid_fortification_scale_up  0.005350

## Conclusions

- Nothing unexpected here!

# 5. VAD and affected causes

In [26]:
deaths = pd.read_hdf(output_dirs[0] + 'deaths.hdf')
deaths.head()

,year,age_group,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,diarrheal_diseases,covered,covered,death,21,baseline,0.0
1,2020,1_to_4,diarrheal_diseases,covered,covered,death,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,diarrheal_diseases,covered,covered,death,21,iron_folic_acid_fortification_scale_up,0.0
3,2020,1_to_4,diarrheal_diseases,covered,covered,death,21,vitamin_a_fortification_scale_up,0.0
4,2020,1_to_4,diarrheal_diseases,covered,covered,death,29,baseline,0.0


In [27]:
np.unique(deaths['cause'])

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'neural_tube_defects', 'other_causes'], dtype=object)

In [28]:
# vad should affect LRI, measles, DD, but NOT NTDs

In [29]:
def get_csmrs(strata_cols):
    deaths = pd.read_hdf(output_dirs[0] + 'deaths.hdf')
    pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')
    deaths = deaths.groupby((['scenario','input_draw'] + strata_cols + ['cause'])).sum()
    pt = pt.groupby((['scenario','input_draw'] + strata_cols)).sum()
    csmrs = deaths / pt
    csmrs = csmrs.reset_index()
    csmrs = csmrs.loc[csmrs.scenario != 'iron_fortification_scale_up']
    csmrs = csmrs.groupby((['cause'] + strata_cols + ['scenario'])).mean()
    return csmrs

In [30]:
overall = get_csmrs([])
overall

# looks good!
# CSMR for other causes may be slightly higher in vitamin A scenario because 
# simulants who did not die of affected causes can now die of another cause

input_draw  \
cause                        scenario                                             
diarrheal_diseases           baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
lower_respiratory_infections baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
measles                      baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
neural_tube_defects          baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
other_causes                 baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   

                                                                            value  
cause                        scenario                                              
diarrheal_diseases           baseline                                3.879801e-06  
                             folic_acid_fortification_scale_up       3.879801e-06  
                             iron_folic_acid_fortification_scale_up  3.879550e-06  
                             vitamin_a_fortification_scale_up        3.704820e-06  
lower_respiratory_infections baseline                                5.468709e-06  
                             folic_acid_fortification_scale_up       5.468709e-06  
                             iron_folic_acid_fortification_scale_up  5.468332e-06  
                             vitamin_a_fortification_scale_up        5.401777e-06  
measles                      baseline                                5.177461e-07  
                             folic_acid_fortification_scale_up       5.177461e-07  
                             iron_folic_acid_fortification_scale_up  5.177461e-07  
                             vitamin_a_fortification_scale_up        4.809896e-07  
neural_tube_defects          baseline                                1.080598e-08  
                             folic_acid_fortification_scale_up       1.080598e-08  
                             iron_folic_acid_fortification_scale_up  1.080598e-08  
                             vitamin_a_fortification_scale_up        1.080736e-08  
other_causes                 baseline                                2.688099e-05  
                             folic_acid_fortification_scale_up       2.688087e-05  
                             iron_folic_acid_fortification_scale_up  2.687220e-05  
                             vitamin_a_fortification_scale_up        2.688424e-05

In [31]:
by_age = get_csmrs(['age_group'])
by_age

# this is behaving as expected
# lower CSMRs for postneonatal and 1-4 age group, but not neonatal age groups

input_draw  \
cause                        age_group      scenario                                             
diarrheal_diseases           1_to_4         baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             early_neonatal baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             late_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             post_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
lower_respiratory_infections 1_to_4         baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             early_neonatal baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             late_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             post_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
...                                                                                        ...   
neural_tube_defects          1_to_4         iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             early_neonatal baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             late_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortif

In [32]:
by_year = get_csmrs(['year'])
by_year

# this is behaving as we would expect with the CORRECT population coverage algorithm (no change until 2020)

input_draw  \
cause                        year scenario                                             
diarrheal_diseases           2020 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2021 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2022 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2023 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
lower_respiratory_infections 2020 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2021 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2022 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2023 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
...                                                                              ...   
neural_tube_defects          2020 iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2021 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2022 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2023 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                          

## Conclusions

This is looking as expected (VAD prevalence appears to be affecting CSMRs due to diarrheal diseases, measles, and LRI.

However, it also appears to be affecting "other causes" which is unexpected